# BrainTrust Shared Drive Setup

This notebook helps you:
1. Connect to the **Life with AI** Shared Drive
2. Auto-discover folder IDs for the BrainTrust migration
3. Verify service account permissions

**Run cells in order from top to bottom.**

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("\n" + "="*50)
print("Google Drive mounted successfully!")
print("="*50)

## Step 2: Initialize Google APIs

This cell sets up the `drive_service` with **Shared Drive support**.

In [ ]:
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# =============================================================================
# CONFIGURATION - Update these paths if needed
# =============================================================================

# Life with AI Shared Drive ID (from the URL)
SHARED_DRIVE_ID = '0AMpJ2pkSpYq-Uk9PVA'

# Possible credential file locations (will try in order)
CREDENTIAL_PATHS = [
    '/content/drive/Shareddrives/Life with AI/credentials.json',
    '/content/drive/Shareddrives/Life with AI/BrainTrust/credentials.json',
    '/content/credentials.json',  # Uploaded directly to Colab
    '/content/drive/MyDrive/credentials.json',  # Personal drive fallback
]

# Scopes needed for Drive and Docs access
SCOPES = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/documents'
]

# =============================================================================
# Find credentials file
# =============================================================================
creds_path = None
for path in CREDENTIAL_PATHS:
    if os.path.exists(path):
        creds_path = path
        print(f"Found credentials at: {path}")
        break

if not creds_path:
    print("ERROR: credentials.json not found!")
    print("\nSearched in:")
    for path in CREDENTIAL_PATHS:
        print(f"  - {path}")
    print("\nPlease either:")
    print("  1. Upload credentials.json to one of the paths above, OR")
    print("  2. Upload directly to Colab using the file browser (left sidebar)")
    raise FileNotFoundError("credentials.json not found")

# =============================================================================
# Authenticate and build services
# =============================================================================
creds = service_account.Credentials.from_service_account_file(
    creds_path, scopes=SCOPES
)

drive_service = build('drive', 'v3', credentials=creds)
docs_service = build('docs', 'v1', credentials=creds)

print("\n" + "="*50)
print("Google APIs initialized successfully!")
print(f"  - drive_service: ready")
print(f"  - docs_service: ready")
print(f"  - Shared Drive ID: {SHARED_DRIVE_ID}")
print("="*50)

## Step 3: Verify Shared Drive Access

Confirms the service account can see the Shared Drive.

In [ ]:
# List all Shared Drives the service account can access
print("Checking Shared Drive access...\n")

try:
    shared_drives = drive_service.drives().list().execute()
    drives_list = shared_drives.get('drives', [])
    
    if not drives_list:
        print("WARNING: No Shared Drives found!")
        print("\nThe service account needs to be added as a member of the Shared Drive.")
        print("Go to Drive > Shared Drives > 'Life with AI' > Manage members")
        print("Add the service account email with 'Content Manager' or 'Manager' role.")
    else:
        print(f"Found {len(drives_list)} Shared Drive(s):\n")
        target_found = False
        for sd in drives_list:
            marker = " <-- TARGET" if sd['id'] == SHARED_DRIVE_ID else ""
            if sd['id'] == SHARED_DRIVE_ID:
                target_found = True
            print(f"  {sd['name']}")
            print(f"    ID: {sd['id']}{marker}")
        
        if target_found:
            print("\n" + "="*50)
            print("SUCCESS: Target Shared Drive is accessible!")
            print("="*50)
        else:
            print(f"\nWARNING: Target drive ID '{SHARED_DRIVE_ID}' not found in list.")
            print("Update SHARED_DRIVE_ID in Step 2 with one of the IDs above.")

except Exception as e:
    print(f"ERROR checking Shared Drives: {e}")

## Step 4: Auto-Discover Folder IDs

Lists all folders in the **Life with AI** Shared Drive.

Copy these IDs to your `FOLDER_IDS` configuration.

In [ ]:
# Query folders ONLY from the Shared Drive
print(f"Discovering folders in Shared Drive...\n")
print("="*60)

try:
    results = drive_service.files().list(
        q="mimeType = 'application/vnd.google-apps.folder' and trashed = false",
        corpora='drive',                    # Search within a specific drive
        driveId=SHARED_DRIVE_ID,            # The Shared Drive ID
        includeItemsFromAllDrives=True,     # Required for Shared Drives
        supportsAllDrives=True,             # Required for Shared Drives
        pageSize=100,
        fields="files(id, name, parents)"
    ).execute()

    folders = results.get('files', [])

    if not folders:
        print("No folders found in the Shared Drive!")
        print("\nPossible issues:")
        print("  - Shared Drive is empty")
        print("  - Service account doesn't have sufficient permissions")
    else:
        print(f"DISCOVERED {len(folders)} FOLDERS\n")
        print("Copy these IDs to your FOLDER_IDS configuration:\n")
        print("-" * 60)
        
        # Sort alphabetically for easier reading
        for folder in sorted(folders, key=lambda x: x['name'].lower()):
            print(f"'{folder['name']}':")
            print(f"    ID: {folder['id']}")
        
        print("-" * 60)
        print(f"\nTotal: {len(folders)} folders")

except Exception as e:
    print(f"ERROR discovering folders: {e}")

## Step 5: Generate FOLDER_IDS Config

Generates a Python dictionary you can copy-paste into your code.

In [ ]:
# Generate copy-paste ready config
print("FOLDER_IDS Configuration (copy this to your code):\n")
print("="*60)
print("FOLDER_IDS = {")

try:
    results = drive_service.files().list(
        q="mimeType = 'application/vnd.google-apps.folder' and trashed = false",
        corpora='drive',
        driveId=SHARED_DRIVE_ID,
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        pageSize=100,
        fields="files(id, name)"
    ).execute()

    folders = results.get('files', [])
    
    for folder in sorted(folders, key=lambda x: x['name'].lower()):
        # Create a clean key name (lowercase, underscores)
        key_name = folder['name'].lower().replace(' ', '_').replace('-', '_')
        print(f"    '{key_name}': '{folder['id']}',  # {folder['name']}")
    
    print("}")
    print("="*60)

except Exception as e:
    print(f"ERROR: {e}")

## Step 6: List Files in a Specific Folder (Optional)

Use this to explore contents of a specific folder.

In [ ]:
# Set the folder ID you want to explore
FOLDER_TO_EXPLORE = ''  # Paste a folder ID here

if not FOLDER_TO_EXPLORE:
    print("Set FOLDER_TO_EXPLORE above to a folder ID from Step 4")
else:
    try:
        results = drive_service.files().list(
            q=f"'{FOLDER_TO_EXPLORE}' in parents and trashed = false",
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            pageSize=50,
            fields="files(id, name, mimeType)"
        ).execute()

        items = results.get('files', [])
        
        if not items:
            print("Folder is empty or not accessible.")
        else:
            # Separate folders and files
            subfolders = [i for i in items if i['mimeType'] == 'application/vnd.google-apps.folder']
            files = [i for i in items if i['mimeType'] != 'application/vnd.google-apps.folder']
            
            print(f"Contents of folder ({len(subfolders)} folders, {len(files)} files):\n")
            
            if subfolders:
                print("FOLDERS:")
                for item in sorted(subfolders, key=lambda x: x['name']):
                    print(f"  [Folder] {item['name']}")
                    print(f"           ID: {item['id']}")
            
            if files:
                print("\nFILES:")
                for item in sorted(files, key=lambda x: x['name']):
                    # Shorten mime type for display
                    mime_short = item['mimeType'].replace('application/vnd.google-apps.', 'g-')
                    print(f"  [{mime_short}] {item['name']}")
                    print(f"           ID: {item['id']}")

    except Exception as e:
        print(f"ERROR: {e}")

## Troubleshooting

**"No Shared Drives found"**
- The service account email needs to be added to the Shared Drive
- Go to Drive > Shared Drives > Right-click "Life with AI" > Manage members
- Add the service account email (found in credentials.json under `client_email`)
- Give it "Content Manager" or "Manager" role

**"credentials.json not found"**
- Upload the file to the Shared Drive at the expected path
- Or upload directly to Colab using the left sidebar file browser

**"Access denied" errors**
- Check the service account has sufficient permissions on the Shared Drive
- Verify the Shared Drive ID is correct